In [1]:
# python
import numpy as np
import pandas as pd
# local
from utils.m4 import smape, mase, M4DatasetGenerator
from utils.ml import print_num_weights
from utils.transformers import get_model
# Neural models
from neuralforecast.core import NeuralForecast
from neuralforecast.models import VanillaTransformer, Informer, Autoformer, FEDformer, PatchTST


## Test Transfered Models

In [2]:
model_name = 'VanillaTransformer'
input_factor = 1
iter = 0

train_all_df = pd.read_parquet('train_all_df.parquet')
nf = NeuralForecast.load(path=f'./trained_models/{model_name}{input_factor}_iter{iter}/')
pred_y_all_predictions = nf.predict(train_all_df)

Seed set to 1


Predicting DataLoader 0: 100%|██████████| 391/391 [00:24<00:00, 15.94it/s]


In [5]:
full_smapes = []
full_mases = []
for run_sp in ['Hourly', 'Daily', 'Weekly', 'Monthly', 'Quarterly', 'Yearly']:
    m4_data = M4DatasetGenerator([run_sp])
    #
    metrics_table = {'serie_id':list(),'smape':list(),'mase':list()}

    for train_serie, test_serie, serie_id, fh, freq, serie_sp in m4_data.get(random=False):    
        pred_y = pred_y_all_predictions.loc[serie_id][model_name].values
        #
        pred_y = pred_y[:fh] # transfered window is less or equal
        #
        serie_smape = smape(test_serie, pred_y)*100
        serie_mase = mase(train_serie, test_serie, pred_y, freq)
        #
        metrics_table['serie_id'].append(serie_id)
        metrics_table['smape'].append(float(serie_smape))
        metrics_table['mase'].append(float(serie_mase))
        full_smapes.append(float(serie_smape))
        full_mases.append(float(serie_mase))

    metrics_dict = {
        'smape_mean': np.round(np.mean(metrics_table['smape'], dtype=float), 3), 
        'mase_mean':  np.round(np.mean(metrics_table['mase'], dtype=float), 3),
    }
    print(run_sp)
    for k, v in metrics_dict.items(): print(f'      {k}: {v}')


Loading M4 Data...
Loaded:
    => Hourly has 414 series
Hourly
      smape_mean: 33.956
      mase_mean: 10.07
Loading M4 Data...
Loaded:
    => Daily has 4227 series
Daily
      smape_mean: 2.993
      mase_mean: 3.186
Loading M4 Data...
Loaded:
    => Weekly has 359 series
Weekly
      smape_mean: 8.281
      mase_mean: 2.359
Loading M4 Data...
Loaded:
    => Monthly has 48000 series


## Métricas finais (Todos os periodos)

In [4]:
metrics_dict = {
        'smape_mean': np.round(np.mean(full_smapes, dtype=float), 3), 
        'mase_mean':  np.round(np.mean(full_mases, dtype=float), 3),
    }
for k, v in metrics_dict.items(): print(f'      {k}: {v}')

      smape_mean: 33.956
      mase_mean: 10.07
